# Téléchargement en utilisant la console

Ce [bloc-notes jupyter](https://jupyter.org) présente l'utilisation de [Sarracenia version 3](https://metpx.github.io/sarracenia) à partir de la ligne de commande (principalement avec Linux, mais devrait être similaire avec Windows et Mac, la principale différence étant des conventions différentes pour l'emplacement de stockage des préférences et des sorties d'éxécution. C'est probablement la façon la plus simple de travailler avec Sarracenia. Vous configurez un flux pour télécharger des fichiers dans un répertoire, et vous pouvez lire le répertoire pour y traiter les fichiers.

In [ ]:
import sarracenia
!mkdir -p ~/.config/sr3/subscribe
!mkdir -p ~/.cache/sr3/log


## Prérequis

Ce code qui précède n'est qu'un moyen d'obtenir des bloc-notes jupyter pour installer metpx-sr3 sur un serveur.
Créer des répertoires au cas où les gens utiliseraient l'accès à l'API sans exécuter les choses via l'API. Le prérequis de base est d'avoir installé metpx-sr3 d'une manière ou d'une autre, soit en tant que package .deb, soit en utilisant pip (ou pip3) disponible pour l'environnement utilisé par jupyter.

Le reste de ce bloc-notes suppose que [metpx-sr3](https://metpx.github.io/sarracenia) est installé.

## SR3

L'interface de ligne de commande s'appelle [sr3](../Reference/sr3.1.html) (abréviation de Sarracenia version 3). On définit les
flux à exécuter à l'aide de fichiers de configuration dans un format simple : format  _keyword_ _value_  (mot clé, valeur).
Voici des exemples de configurations pour vous aider à démarrer:

In [3]:
!sr3 list examples

Sample Configurations: (from: /home/peter/Sarracenia/sr3/sarracenia/examples )
cpump/cno_trouble_f00.inc        poll/airnow.conf                 
poll/aws-nexrad.conf             poll/mail.conf                   
poll/nasa-mls-nrt.conf           poll/noaa.conf                   
poll/soapshc.conf                poll/usgs.conf                   
post/WMO_mesh_post.conf          sarra/wmo_mesh.conf              
sender/ec2collab.conf            sender/pitcher_push.conf         
shovel/no_trouble_f00.inc        subscribe/aws-nexrad.conf        
subscribe/dd_2mqtt.conf          subscribe/dd_all.conf            
subscribe/dd_amis.conf           subscribe/dd_aqhi.conf           
subscribe/dd_cacn_bulletins.conf subscribe/dd_citypage.conf       
subscribe/dd_cmml.conf           subscribe/dd_gdps.conf           
subscribe/dd_radar.conf          subscribe/dd_rdps.conf           
subscribe/dd_swob.conf           subscribe/ddc_cap-xml.conf       
subscribe/ddc_normal.conf        subscribe/downloa

Il existe différents types de flux : les exemples sont classés selon le type de flux (poll, post, sarra, sender, shovel...)
Un _subscribe_ (abonnement) est utilisé par les clients pour télécharger à partir d'une pompe de données. Choisissons-en un.

In [4]:
!sr3 add subscribe/hpfx_amis.conf

add: 2022-03-19 13:17:47,786 2724188 [INFO] sarracenia.sr add copying: /home/peter/Sarracenia/sr3/sarracenia/examples/subscribe/hpfx_amis.conf to /home/peter/.config/sr3/subscribe/hpfx_amis.conf 



Les fichiers qui sont actifs pour vous sont placés dans ~/.config/sr3/<flow_type>/config_name. Vous pouvez y naviguer
et les modifier avec un éditeur si vous le souhaitez. Vous pouvez également le faire avec _sr3 edit subscribe/hpfx_amis.conf_

    # this is a feed of wmo bulletin (a set called AMIS in the old times)

    broker amqps://hpfx.collab.science.gc.ca/
    exchange xpublic

    # instances: number of downloading processes to run at once.  defaults to 1. Not enough for this case
    instances 5
   
    # expire, in operational use, should be longer than longest expected interruption
    expire 10m

    topic_prefix v02.post
    subtopic *.WXO-DD.bulletins.alphanumeric.#
    mirror false
    directory /tmp/amis/
    accept .*

ajouté messageCountMax, donc il ne s'exécute pas éternellement.

In [6]:
!mkdir /tmp/amis
!echo messageCountMax 10 >>~/.config/sr3/subscribe/hpfx_amis.conf

Le répertoire racine où les fichiers doivent être placés doit exister avant de commencer.
les commandes ci-dessus sont à configurer sur une machine Linux, vous pourriez avoir besoin d'autre chose sur un Mac ou Windows.

Vous pouvez ensuite exécuter un flux de manière interactive avec l'action _foreground_, et il se terminera rapidement, comme ceci:

In [7]:
!sr3 foreground subscribe/hpfx_amis.conf

2022-03-19 13:18:34,230 2724487 [INFO] sarracenia.config fill_missing_options overriding batch for consistency with messageCountMax: 10
.2022-03-19 13:18:34,442 [INFO] 2724489 sarracenia.config fill_missing_options overriding batch for consistency with messageCountMax: 10
2022-03-19 13:18:34,444 [INFO] 2724489 sarracenia.config fill_missing_options overriding batch for consistency with messageCountMax: 10
2022-03-19 13:18:34,444 [INFO] 2724489 sarracenia.flow loadCallbacks plugins to load: ['sarracenia.flowcb.gather.message.Message', 'sarracenia.flowcb.retry.Retry', 'sarracenia.flowcb.housekeeping.resources.Resources', 'sarracenia.flowcb.log.Log']
2022-03-19 13:18:34,803 [INFO] 2724489 sarracenia.flowcb.log __init__ subscribe initialized with: {'on_housekeeping', 'after_accept', 'after_work'}
2022-03-19 13:18:34,803 [INFO] 2724489 sarracenia.flow run options:
_Config__admin="...st/ None True True False False None None",
_Config__broker="...ca/ None True True False False None None",
_Co

comme vous pouvez le voir, il a téléchargé cinq fichiers dans /tmp/amis.
L'action _foreground_ est destinée à aider au débogage, plutôt qu'aux opérations réelles.

In [8]:
!sr3 status

2022-03-19 13:18:52,445 2724517 [INFO] sarracenia.config fill_missing_options overriding batch for consistency with messageCountMax: 10
status: 
Component/Config                         State        Run  Miss   Exp Retry
----------------                         -----        ---  ----   --- -----
subscribe/hpfx_amis                      stopped        0     0     0     0
      total running configs:   0 ( processes: 0 missing: 0 stray: 0 )


Il y a 1 configuration dans votre liste. Vous pouvez en avoir des centaines. Les colonnes de droite indiquent le nombre d'instances dont vous disposez pour chaque configuration. Dans l'exemple ci-dessus, _instances_ est défini sur 5, donc on s'attendrait à voir 5 instances en cours d'exécution lors de l'exécution. Vous pouvez démarrer une configuration spécifique avec _sr3 start subscribe/*_ ou démarrer toutes les instances actives avec : _sr3 start_

In [ ]:
!sr3 log subscribe/hpfx_amis.conf

2022-03-19 13:18:58,254 2724529 [INFO] sarracenia.config fill_missing_options overriding batch for consistency with messageCountMax: 10
2022-03-19 11:10:07,282 [INFO] sarracenia.flowcb.log after_work downloaded ok: /tmp/hpfx_amis/SOVX45_KWAL_191509___38991 
2022-03-19 11:10:07,282 [INFO] sarracenia.flowcb.log after_work downloaded ok: /tmp/hpfx_amis/SRCN40_KWAL_191509___52945 
2022-03-19 11:10:07,282 [INFO] sarracenia.flowcb.log after_work downloaded ok: /tmp/hpfx_amis/SXCN40_KWAL_191509___11643 
2022-03-19 11:10:07,282 [INFO] sarracenia.flowcb.log after_work downloaded ok: /tmp/hpfx_amis/SRCN40_KWAL_191509___30237 
2022-03-19 11:10:07,738 [INFO] sarracenia.flowcb.log after_accept accepted: (lag: 12.03 ) https://hpfx.collab.science.gc.ca /20220319/WXO-DD/bulletins/alphanumeric/20220319/SO/KWNB/15/SOVD83_KWNB_191200_RRX__37893 
2022-03-19 11:10:07,898 [INFO] sarracenia.flowcb.log after_work downloaded ok: /tmp/hpfx_amis/SOVD83_KWNB_191200_RRX__37893 
2022-03-19 11:10:17,165 [INFO] root 

Lors de l'exécution en arrière-plan, la sortie doit aller dans un fichier journal (sortie d'exécution). Comme nous n'avons exécuté ce fichier de configuration qu'au premier plan, demander à voir le journal imprime une erreur indiquant que le journal est manquant. Cela vous indique que les journaux se trouvent dans le répertoire _~/.cache/sr3/log_. Les journaux peuvent être surveillés en temps réel avec des outils traditionnels tels que _tail -f_ ou _grep_.

_sr3 stop_ fait ce que vous pensez.

Les processus peuvent planter. Dans la sortie _sr3 status_ ci-dessus, si le nombre de processus dans la colonne Run est inférieur à celui dans la colonne Exp (for Expected), cela signifie que certaines instances ont planté. Vous pouvez le réparer (démarrez simplement les instances manquantes) avec :

_sr3 sanity_ - démarre les instances manquantes, tue également les parasites s'il en trouve.

Voilà, une introduction à l'exécution des configurations dans Sarracenia à partir de la ligne de commande.


## Conclusion

Si tout ce que vous voulez faire est d'obtenir des données à partir d'une pompe de données en temps réel, utiliser l'interface de ligne de commande pour contrôler certains processus qui s'exécutent tout le temps, afin qu'ils vident les fichiers dans un certain répertoire est la méthode la plus simple.

Ce n'est pas très efficace cependant. Lorsque vous avez un grand nombre de fichiers sur lesquels travailler et que vous souhaitez un traitement à grande vitesse, il est préférable, dans le sens d'une charge CPU et d'E/S (I/O) réduite et en termes de vitesse de traitement
d'avoir votre propre application informée de l'arrivée des fichiers, plutôt que de scanner un répertoire.

La façon la plus simple de le faire est d'ajouter des rappels à vos flux. Nous couvrirons cela ensuite.